#### Load Packages

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import h5py
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import rcParams
from scipy import stats
from scipy.stats import mannwhitneyu
import statsmodels.stats.multitest as smt
import csv
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Import Utility Functions
sys.path.append("/content/drive/MyDrive") # Change utility_functions.py file path to local location
from  utils import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import rcParams

import seaborn as sns
sns.set_style('white')

# The snippet below is for better visualization
font_list = []
fpaths = matplotlib.font_manager.findSystemFonts()
for i in fpaths:
    try:
        f = matplotlib.font_manager.get_font(i)
        font_list.append(f.family_name)
    except RuntimeError:
        pass

font_list = set(font_list)
plot_font = 'Helvetica' if 'Helvetica' in font_list else 'FreeSans'

rcParams['font.family'] = plot_font
rcParams.update({'font.size': 10})

rcParams.update({'figure.dpi': 300})
rcParams.update({'figure.figsize': (5,5)})
rcParams.update({'savefig.dpi': 300})
rcParams.update({'axes.labelsize': 10})

import warnings
warnings.filterwarnings('ignore')

#### Data Loading, QC, Normalization

Construct expression dataframes including foci and features of interest.

In [4]:
# Read the data (.csv file) into dataframe
file_path = "/content/drive/My Drive/20220914_MCF7_IR_merged_annotated.csv" # change file path to local location
data = pd.read_csv(file_path)

In [5]:
data['guide'] = data['guide'].astype(str)
data["ID"] = data.index

In [6]:
# Construct expression df
cols = ["ID", "guide", "RAD51_foci", "BRCA1_foci", "X53BP1_foci", "yH2AX_foci", "micronuclei", "cell_cycle"]
expr_df = data.loc[:, cols]
expr_df.head()

# Normalize expression per foci for all foci across guides - Not normalized for KS and Wasserstein
# expr_df[["RAD51_foci", "BRCA1_foci", "X53BP1_foci", "yH2AX_foci", "micronuclei"]] = expr_df[["RAD51_foci", "BRCA1_foci", "X53BP1_foci", "yH2AX_foci", "micronuclei"]].apply(lambda x: (x-x.mean()) / np.std(x, ddof=1), axis=0)


,ID,guide,RAD51_foci,BRCA1_foci,X53BP1_foci,yH2AX_foci,micronuclei,cell_cycle
0,0,AAVS1.controls.AAVS1.100,0,0,1,2,0,G1
1,1,AAVS1.controls.AAVS1.100,6,5,2,4,0,G2/S
2,2,AAVS1.controls.AAVS1.100,0,3,1,2,0,G1
3,3,AAVS1.controls.AAVS1.100,16,17,0,1,0,G2/S
4,4,AAVS1.controls.AAVS1.100,5,6,2,3,1,G2/S


#### Define Non-Targeting, AAVS1 and Perturbation Lists
Define three lists containning guides under Non-Targeting Control, AAVS1 Control and Perturbations. In this dataset, there should be:

*   Non-Targeting Controls: 34 unique guides
*   AAVS1 Controls: 32 unique guides
*   Perturbations: 275 unique guides


In [7]:
# Initiate three empty lists for Non-Targeting(NT), AAVS1(AAVS), and Perturbations(Perturb)
NT_list = []
AAVS_list = []
perturb_list = []

# Loop through all guides and append guides under each associated class
for i in expr_df["guide"]:
    if "targeting" in i:
        NT_list.append(str(i))
    elif "AAVS" in i:
        string_list = str(i).split(".") # Change the original guide name (AAVS1.controls.AAVS1.###) to AAVS1.###
        AAVS_list.append(".".join([string_list[2],  string_list[3]]))
    else:
        string_list = str(i).split(".") # Change the original guide name (Core.Perturbation.###) to Perturbation.###
        perturb_list.append(".".join([string_list[1],  string_list[2]]))

# Store unique guides in each class and their frequencies (number of cells under that guide) in dictionaries
NT_freq = Counter(NT_list)
NT_dict = dict(NT_freq)

AAVS_freq = Counter(AAVS_list)
AAVS_dict = dict(AAVS_freq)

perturb_freq = Counter(perturb_list)
perturb_dict = dict(perturb_freq)

# Define lists for unique for Non-Targeting(NT), AAVS1(AAVS), and Perturbations(Perturb) guides
NT_list = np.unique(NT_list).tolist()
AAVS_list = np.unique(AAVS_list).tolist()
perturb_list = np.unique(perturb_list).tolist()

#### Separation by Cell Cycle *Optional*

*   Motivation: In the visualization notebook, we show that there seems to be foci expression separation between G1 and G2/S cells. In particular, this is observed for RAD51 and BRCA1 foci expression. Therefor, it may make sense to separate the foci expression analysis for G1 and G2/S cells.
*   The code snippet below splits the expression dataframe based on cell cycle. Simply input the cell cycle of interest to the following statistical analysis.



In [8]:
# Binarize the cell_cycle feature
expr_df["cell_cycle"] = expr_df["cell_cycle"].astype(str)
expr_df["cell_cycle"] = expr_df["cell_cycle"].apply(lambda x: cc_binary(x))

# Split the expression dataframe based on the 'cell_cycle' column
split_data = expr_df.groupby('cell_cycle')

# Access the individual groups
G1_df = split_data.get_group(0)
G2_df = split_data.get_group(1)

# G1_df.head()

,ID,guide,RAD51_foci,BRCA1_foci,X53BP1_foci,yH2AX_foci,micronuclei,cell_cycle
0,0,AAVS1.controls.AAVS1.100,0,0,1,2,0,0
2,2,AAVS1.controls.AAVS1.100,0,3,1,2,0,0
5,5,AAVS1.controls.AAVS1.100,0,0,0,3,0,0
7,7,AAVS1.controls.AAVS1.100,0,1,14,18,0,0
8,8,AAVS1.controls.AAVS1.100,5,5,2,1,0,0


#### Conducting Two-Sample KS Test Compute Wasserstein Distance
The following loop iterates through the foci (feature) list and for each foci *i*, the loop iterates through the perturbation list and for each perturbation *j*, computes:

*   The Two-Sample KS Test Statistic of foci *i* expression of control (NT or AAVS1 or combined) population v.s. perturbation *j* population.
*   The Two-Sample KS Test p-value of foci *i* expression of control (NT or AAVS1 or combined) population v.s. perturbation *j* population. The p-value is FDR-adjusted using the Benjamini-Hochberg Procedure.
*   The Wasserstein Distance between foci *i* expression of control (NT or AAVS1 or combined) population and perturbation *j* population.

It is possible to integrate the MWU function into this loop. The loop outputs three dataframes containning the three different values it computes. The dataframes are foci (rows) by perturbations (columns). ie. The *i*th row and *j*th column of the foci_by_gene_p dataframe contains the Two-Sample KS FDR-Adjusted p-value of foci *i* expression of control v.s. perturbation *j*.

In [9]:
# Define the Foci list and the Perturbation list
foci = ["RAD51_foci", "BRCA1_foci", "X53BP1_foci", "yH2AX_foci", "micronuclei"]

test_perturb_list = AAVS_list + perturb_list # Here, we are testing AAVS1 controls as pseudo-perturbations against non-targeting controls. Can set this as containning strictly perturbations.
perturbations = [] # perturbations is a list containning all guides we are testing against defined control. Here it contains both AAVS1 and perturbation guides.
for per in test_perturb_list:
    perturbations.append(per)

# Initiate dataframes to store KS p-values, KS test statistics and Wasserstein distances.
foci_by_gene_p = pd.DataFrame(columns=perturbations)
foci_by_gene_stat = pd.DataFrame(columns=perturbations)
foci_by_gene_dist = pd.DataFrame(columns=perturbations)

# Nested loop iterating through foci and perturbations.
for idx, i in enumerate(foci):
    p_values = []
    KS_stats = []
    WS_dist = []

    for jdx, j in enumerate(test_perturb_list):
        dataframe = guide_df("targeting", str(j), expr_df) # Optionally can analyze G1 and G2/S separately, additonally may choose to define a different control (here we used non-targeting controls).
        dataframe["guide"] = dataframe["guide"].apply(lambda x: binary(x, str(j)))

        KS_stat, KS_p = KS(i, dataframe)
        WS_d = WS(i, dataframe)

        p_values.append(KS_p)
        KS_stats.append(KS_stat)
        WS_dist.append(WS_d)

    foci_by_gene_p.loc[str(i)] = smt.multipletests(p_values, method="fdr_bh")[1] # Implementing the Benjamini-Hochberg FDR correction
    foci_by_gene_stat.loc[str(i)] = KS_stats
    foci_by_gene_dist.loc[str(i)] = WS_dist

del idx, i, jdx, j

# foci_by_gene_p.head()
# foci_by_gene_stat.head()
# foci_by_gene_dist.head()

In [10]:
# # Write the dataframes into CSV files
# foci_by_gene_p.to_csv('foci_by_gene_p.csv', index=False)
# foci_by_gene_stat.to_csv('foci_by_gene_stat.csv', index=False)
# foci_by_gene_dist.to_csv('foci_by_gene_dist.csv', index=False)

In [11]:
# For each perturbation guide, find all KS-significant foci (p-value < 0.05), store the significant foci, its associated p-value and test statistic into a dictionary.
gene_foci_dict = {} # Initiate empty dictionary to store values in nested list: ie. 'guide.xxx': [['significant_foci_A', p-value, test statistic], ['significant_foci_B', p-value, test statistic], ...]

for jdx, j in enumerate(test_perturb_list):
    gene_foci_dict[str(j)] = []

    for idx, i in enumerate(foci):
        dataframe = guide_df("targeting", str(j), expr_df)
        dataframe["guide"] = dataframe["guide"].apply(lambda x: binary(x, str(j)))

        if foci_by_gene_p.loc[str(i), str(j)] < 0.05:
            gene_foci_dict[str(j)].append([str(i), foci_by_gene_p.loc[str(i), str(j)], foci_by_gene_stat.loc[str(i), str(j)]])

# print(gene_foci_dict)

In [12]:
# # Write the dictionary into a CSV file
# def write_dict_to_csv(dictionary, file_path):
#     with open(file_path, 'w', newline='') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=dictionary.keys())
#         writer.writeheader()
#         writer.writerow(dictionary)
#     print("Dictionary successfully written to CSV file.")

# write_dict_to_csv(gene_foci_dict, 'gene_foci_dict.csv') # May change the file path and name.